# Notebook 기본 세팅

In [1]:
# Constant 선언

# 프로젝트 루트 디렉토리를 식별하기 위한 마커 파일 이름
ROOT_MARKER = "pyproject.toml"

# 한글 표시를 위한 나눔바른고딕 폰트 파일 이름
# matplotlib 의 font_manager 에 실제 폰트 파일의 위치를 넣어주어야 한다.
KOREAN_FONT_FILE = "NanumBarunGothic.ttf"

# matplotlib 에서는 font-family 의 이름으로 font 를 설정한다.
# 그래서 font 파일 그 자체가 아니라, 그 파일의 family 이름을 적어준다.
KOREAN_FONT_FAMILY = "NanumBarunGothic"

# 참고
# Font Family 와 Font File 의 차이는,
# Font Family 는 비슷한 디자인 특성을 공유하는 글꼴 그룹을 의미한다.
#
# 예를 들어 '나눔바른고딕' 폰트 패밀리는 일반(Regular), 굵게(Bold), 기울임(Italic) 등 여러 스타일을 포함할 수 있다.
# 반면, 폰트 파일(.ttf, .otf 등)은 이러한 폰트의 하나의 스타일이 저장된 실제 파일이다.
#
# 이 프로젝트에서는 폰트 용량을 줄이기 위해 일반(Regular) 인 NanumBarunGothic.ttf 만 사용한다.

In [2]:
# 프로젝트 root 를 sys.path 에 추가해서 import 구문을 사용하기 쉽게
from pathlib import Path


def find_project_root() -> Path:
    """
    pyproject.toml 파일을 기준으로 루트 디렉토리를 찾는다.
    :return: Path: 프로젝트 루트 디렉토리 경로
    """

    current_path = Path().resolve()

    while current_path != current_path.parent:
        if (current_path / ROOT_MARKER).exists():
            return current_path

        current_path = current_path.parent

    raise FileNotFoundError("프로젝트 루트 디렉토리를 찾을 수 없습니다.")


ROOT_DIR = find_project_root()

In [3]:
# matplotlib 의 한글 font 설정
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt


FONTS_DATA_DIR = ROOT_DIR / "notebooks" / "fonts"


def setup_korean_font():
    font_path = FONTS_DATA_DIR / KOREAN_FONT_FILE
    fm.fontManager.addfont(font_path)

    # 폰트 설정
    plt.rcParams["font.family"] = KOREAN_FONT_FAMILY
    plt.rcParams["axes.unicode_minus"] = False


setup_korean_font()

# Train

In [5]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# 1. 데이터 불러오기
df = pd.read_csv("df_prepared.csv")

# 2. 날짜 파싱 및 연도 추출
df['tm'] = pd.to_datetime(df['tm'])
df['year'] = df['tm'].dt.year

# 3. 학습/검증 분리
train_df = df[df['year'] < 2025].copy()
train_df = train_df.dropna(subset=target_cols)
valid_df = df[df['year'] == 2025].copy()
valid_df = valid_df.dropna(subset=target_cols)

# 4. feature / target 분리
features = [col for col in df.columns if not col.startswith('iscs_') and col not in ['tm', 'year', 'iscs']]
target_cols = [col for col in df.columns if col.startswith('iscs_')]

X_train, y_train = train_df[features], train_df[target_cols]
X_valid, y_valid = valid_df[features], valid_df[target_cols]

# 5. 모델 학습 (멀티레이블이므로 각 타겟별로 개별 학습)
rf_models = {}
y_preds = pd.DataFrame(index=X_valid.index)

for target in target_cols:
    clf = RandomForestClassifier(random_state=42, n_estimators=100)
    clf.fit(X_train, y_train[target])
    rf_models[target] = clf
    y_preds[target] = clf.predict(X_valid)

# 6. 평가
for target in target_cols:
    print(f"\n📌 타겟: {target}")
    print("🎯 Accuracy:", accuracy_score(y_valid[target], y_preds[target]))
    print(classification_report(y_valid[target], y_preds[target], zero_division=0))

/tmp/ipykernel_18543/551789270.py:6: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("df_prepared.csv")



📌 타겟: iscs_가루눈
🎯 Accuracy: 1.0
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     10422

    accuracy                           1.00     10422
   macro avg       1.00      1.00      1.00     10422
weighted avg       1.00      1.00      1.00     10422


📌 타겟: iscs_기타
🎯 Accuracy: 0.8578967568604874
              precision    recall  f1-score   support

         0.0       0.54      0.19      0.28      1527
         1.0       0.87      0.97      0.92      8895

    accuracy                           0.86     10422
   macro avg       0.71      0.58      0.60     10422
weighted avg       0.83      0.86      0.83     10422


📌 타겟: iscs_낮은안개
🎯 Accuracy: 0.9994242947610823
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     10416
         1.0       0.00      0.00      0.00         6

    accuracy                           1.00     10422
   macro avg       0.50      0.50      0.50     10422
we